In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy import ndimage

In [2]:
all_tracks = pickle.load(open('all_tracks.pkl'))


In [3]:
def overlaps(box1,box2):
    
    if (box1[2] <= box2[3]+1 and
        box1[3]+1 >= box2[2] and
        box1[4] <= box2[5]+1 and
        box1[5]+1 >= box2[4]):
        diff = np.array(box2[1:3])-np.array(box1[1:3])
        angle = np.arctan2(diff[1],diff[0])*180/3.14159

        if angle < 45.0 or angle > 315.0:
            return 1 #top
        elif angle < 135.0:
            return 2 #bottom
        elif angle < 225.0:
            return 3 #right
        else:
            return 4 #left
        
    else:
        return -1
    

In [4]:
tile_tracks = pickle.load(open('tile_tracks.pkl'))

In [5]:
collisions = {}
(animation_at_time,animation_indices,groups) = pickle.load(open('animation_at_time.pkl','rb'))
(tile_animation_at_time,tile_animation_indices,tile_groups) = pickle.load(open('tile_animation_at_time.pkl','rb'))
ind2anim = {}
for a in tile_animation_indices:
    ind2anim[tile_animation_indices[a]] = a

In [32]:
track2group = {}
for g in groups:
    for t in groups[g]:
        track2group[t] = g

tile2group = {}   
for g in tile_groups:
    for t in tile_groups[g]:
        tile2group[t] = g

collisions = {}  
start_time_out = 5
for track_id,track in enumerate(all_tracks):
    times = sorted(track[1])
    for other_id,other in enumerate(all_tracks):
        if track_id < other_id:
            other_times = sorted(other[1])
            track_collisions = {}
            last_collision = -1
            other_start = other_times[0]
            for t in times[start_time_out:]:
                
                if t in other_times and t > other_start+start_time_out:
                    blob1 = animation_at_time[track[0]][t]
                    blob2 =  animation_at_time[other[0]][t]

                    collision = -1
                    sprite_box = track[1][t][1]
                    other_box = other[1][t][1]
                    collision =  overlaps(sprite_box,other_box)
                    if collision >= 0:
                        if t not in collisions:
                            collisions[t] = []
                        if True:#t-last_collision > 1:
                            collisions[t].append((t,collision,blob1,blob2))
                            collisions[t].append((t,collision,track2group[track[0]],track2group[other[0]]))
                            track_collisions[t] = collision
                        else:
                            print 'skipped'
                        last_collision  = t
                

import re
for track_id,track in enumerate(tile_tracks[:]): 
    times = sorted(track[1])
    match = re.match('tile(\d+),(\d+)',track[0])
    y = int(match.group(1))
    x = int(match.group(2))
    location = (y,x)
    for other_id,other in enumerate(all_tracks):
        other_times = sorted(other[1])
        track_collisions = {}
        last_collision = -1
        other_start = other_times[0]
        for t in times[start_time_out:]:
            
            if t in other_times  and t > other_start+start_time_out and t in tile_animation_at_time[location]:
                blob = ('tile', tile_animation_at_time[location][t])
                collision = -1

                blob2 =  animation_at_time[other[0]][t]

                
                bounding_box = track[1][t][1]
                other_box = other[1][t][1]
                collision =  overlaps(bounding_box,other_box)

                if collision >= 0:
                    if t == 9 and blob == ('tile',52):
                        print location
                    if t not in collisions:
                        collisions[t] = []
                    if True:#t-last_collision > 1:
                        collisions[t].append((t,collision,blob,blob2))
                        collisions[t].append((t,collision,('tilegroup',tile2group[location]),track2group[other[0]]))
                        track_collisions[t] = collision
                    last_collision  = t

(19, 4)
(20, 4)
(21, 4)
(19, 5)
(20, 5)
(21, 5)
(19, 6)
(21, 6)
(20, 7)
(21, 7)


In [67]:
tile_changes = pickle.load(open('tile_changes.pkl'))
change_at_time = {}

for change in tile_changes:
    for t in tile_changes[change]:
        if t not in change_at_time:
            change_at_time[t] = set()
        change_at_time[t].add(change)

In [68]:

events = {}
counts = {}
timesteps = 0.0
cocounts = {}
for t in collisions:
    timesteps += 1.0
    seen = set()
    for coll in collisions[t]:
        non_directional_coll = coll[2:]
        for ii in range(0,3):
            if non_directional_coll not in counts:
                counts[non_directional_coll] = 0.
            counts[non_directional_coll] += 1.
            
            if t in change_at_time:
                for change in change_at_time[t]:
                    if (change,non_directional_coll) in seen:
                        break
                        
                    if change not in cocounts:
                        cocounts[change] = {}
                    if non_directional_coll not in cocounts[change]:
                        cocounts[change][non_directional_coll] = 0.
                    cocounts[change][non_directional_coll] += 1.
                    seen.add((change,non_directional_coll))
                        


In [65]:
for change in sorted(tile_changes):
    print change,tile_changes[change]
    print ind2anim[change[0]], '->', ind2anim[change[1]]
    p1 = float(len(tile_changes[change]))/timesteps
    for co in cocounts[change]:
        p2 = counts[co]/timesteps
        p12 = cocounts[change][co]/timesteps
        pmi = np.log(p12/(p1*p2))
        npmi = -pmi/np.log(p12)
        if npmi > 0.6:
            print cocounts[change][co]
            print ind2anim[change[0]], '->', ind2anim[change[1]], co, npmi,p1,p2,p12,pmi,timesteps
            
    print ''

In [78]:
sprite_changes = pickle.load(open('sprite_changes.pkl'))
change_at_time = {}

for change in sprite_changes:
    for t in sprite_changes[change]:
        if t not in change_at_time:
            change_at_time[t] = set()
        change_at_time[t].add(change)
        
print sprite_changes

{(5, 9): [497, 607, 1377], (22, 23): [708], (23, 24): [835], (7, 3): [1613], (5, 6): [122, 250, 677, 932, 1002, 1200], (10, 6): [722, 1301, 1436], (8, 9): [170], (13, 16): [993, 908], (11, 14): [220], (15, 19): [371], (3, 7): [804], (8, 5): [1175], (7, 8): [168, 1158], (6, 7): [161, 289, 1142, 1226, 1600], (18, 12): [652], (12, 13): [984, 181, 376, 907, 614], (9, 6): [171, 499, 609], (7, 6): [297, 396, 824, 1232, 1386], (6, 10): [342, 426, 529, 707, 1292, 1431], (19, 14): [324], (6, 3): [778], (10, 9): [367], (9, 7): [388, 1379], (6, 4): [241, 645, 896, 988, 1033, 1099, 1336, 1470, 1521], (10, 4): [543], (4, 5): [17, 244, 473, 547, 648, 901, 991, 1339], (3, 4): [9], (11, 12): [927, 98, 910, 319, 754, 545, 1363], (0, 1): [1566], (12, 15): [732, 1004, 908, 988, 396, 364, 620, 1468, 1452], (4, 6): [1035, 1107, 1475, 1526], (16, 13): [916, 628, 1460], (7, 4): [465], (14, 12): [764, 228, 332, 1356], (18, 11): [908], (17, 12): [908, 540, 652], (10, 7): [442], (15, 16): [734, 1009, 915, 992, 

In [79]:

events = {}
counts = {}
timesteps = 0.0
cocounts = {}
for t in collisions:
    timesteps += 1.0
    seen = set()
    for coll in collisions[t]:
        non_directional_coll = coll[2:]
        for ii in range(0,3):
            if non_directional_coll not in counts:
                counts[non_directional_coll] = 0.
            counts[non_directional_coll] += 1.
            
            if t in change_at_time:
                for change in change_at_time[t]:
                    if (change,non_directional_coll) in seen:
                        break
                        
                    if change not in cocounts:
                        cocounts[change] = {}
                    if non_directional_coll not in cocounts[change]:
                        cocounts[change][non_directional_coll] = 0.
                    cocounts[change][non_directional_coll] += 1.
                    seen.add((change,non_directional_coll))
                        

In [82]:
for change in sorted(sprite_changes):
    print change,sprite_changes[change]
    p1 = float(len(sprite_changes[change]))/timesteps
    highest = 0
    best = ()
    for co in cocounts[change]:
        p2 = counts[co]/timesteps
        p12 = cocounts[change][co]/timesteps
        pmi = np.log(p12/(p1*p2))
        npmi = -pmi/np.log(p12)
        if npmi > highest:
            highest = npmi
            best = (change,co, npmi,p1,p2,p12,pmi,timesteps)
        if npmi > 0.6:
            print npmi,(change,co, npmi,p1,p2,p12,pmi,timesteps)
    print 'highest', highest,best
    print ''

(0, 1) [1566]
highest 0 ()

(3, 4) [9]
highest 0.374824704375 ((3, 4), (('tile', 34), 4), 0.3748247043746758, 0.0005847953216374269, 0.06140350877192982, 0.0005847953216374269, 2.7902882993391822, 1710.0)

(3, 7) [804]
highest 0.44999895154 ((3, 7), (('tile', 32), 5), 0.4499989515397349, 0.0005847953216374269, 0.03508771929824561, 0.0005847953216374269, 3.3499040872746049, 1710.0)

(4, 5) [17, 244, 473, 547, 648, 901, 991, 1339]
highest 0.38686247513 ((4, 5), (('tile', 9), 4), 0.38686247513018995, 0.004678362573099415, 0.007017543859649123, 0.0005847953216374269, 2.8799004580288692, 1710.0)

(4, 6) [1035, 1107, 1475, 1526]
highest 0.174064625537 ((4, 6), (('tile', 0), 5), 0.17406462553701302, 0.0023391812865497076, 0.06842105263157895, 0.0005847953216374269, 1.2957803535790589, 1710.0)

(5, 6) [122, 250, 677, 932, 1002, 1200]
highest 0.277928685516 ((5, 6), (('tile', 12), 4), 0.27792868551649202, 0.0035087719298245615, 0.021052631578947368, 0.0005847953216374269, 2.0689702418125404, 17

In [ ]:
print all_tracks[2][1][174]